In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DecimalType, StructType, StructField, IntegerType
from pyspark.sql import Row
import decimal

# Initialize Spark session
spark = SparkSession.builder.appName("GenerateDecimalParquet").getOrCreate()



24/09/19 12:17:05 WARN Utils: Your hostname, Enhance42s-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.214 instead (on interface en0)
24/09/19 12:17:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/19 12:17:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")
spark.sparkContext.setLogLevel("ERROR")
global data1, data2, schema1, schema2

In [28]:
# Create sample data with DecimalType(10,2) for partition 1
def decimaltest(d1,d2):
    global data1, data2, schema1, schema2
    data1 = [
        Row(id=1, value=decimal.Decimal("12345.67")),
        Row(id=2, value=decimal.Decimal("98765.00")),
        Row(id=3, value=decimal.Decimal("98765.43"))
    ]
    data1 = [
        Row(id=1, value=decimal.Decimal("123456789012.67")),
        Row(id=2, value=decimal.Decimal("987656789012.00")),
        Row(id=3, value=decimal.Decimal("987656789012.43"))
    ]
    
    schema1 = StructType([
        StructField("id", IntegerType(), False),
        StructField("value", DecimalType(d1[0], d1[1]), False)
    ])
    
    df1 = spark.createDataFrame(data1, schema1)
    
    # Create sample data with DecimalType(10,0) for partition 2
    data2 = [
        Row(id=4, value=decimal.Decimal("12345.678")),
        Row(id=5, value=decimal.Decimal("98765.432")),
        Row(id=6, value=decimal.Decimal("98765.432"))
    ]
    data2 = [
        Row(id=4, value=decimal.Decimal("123456789012.678")),
        Row(id=5, value=decimal.Decimal("987656789012.432")),
        Row(id=6, value=decimal.Decimal("987656789012.432"))
    ]
    
    schema2 = StructType([
        StructField("id", IntegerType(), False),
        StructField("value", DecimalType(d2[0], d2[1]), False)
    ])
    
    df2 = spark.createDataFrame(data2, schema2)
    
    # Write df1 to partition "part=1" with DecimalType(10,2)
    df1.write.mode("overwrite").parquet("test/part=1")
    
    # Write df2 to partition "part=2" with DecimalType(12,3)
    df2.write.mode("overwrite").parquet("test/part=2")
    #print("dataframe of partition 1:\n", df1.show())
    #print("dataframe of partition 2:\n", df2.show())


In [4]:
# Verify the written parquet files .schema(schema2)
def readpartitions(inpschema):
    try:
        df_read = spark.read.schema(inpschema).parquet("test")
        #print(df_read.printSchema())
        df_read.show()
    except Exception as e:
        print("read partition exception")
        raise
def readpartitions_mergeschema():
    try:
        df_read = spark.read.option("mergeSchema", "true").parquet("test")
        #print(df_read.printSchema())
        df_read.show()
    except Exception as e:
        print("read partition with merge schema exception")
        raise   

def readpartitions_withoutschema():
    try:
        df_read = spark.read.parquet("test")
        #print(df_read.printSchema())
        df_read.show()
    except Exception as e:
        print("read partition without merge schema exception")
        raise  

### Decimal Type 
int32: for 1 <= precision <= 9
int64: for 1 <= precision <= 18; precision < 10 will produce a warning

scale changes 

In [5]:
# Verify the written parquet files   .schema(schema1)  .option("mergeSchema", "true")
df_read1 = spark.read.parquet("test/part=1")
df_read1 = df_read1.withColumn("value", df_read1["value"].cast(DecimalType(8, 2)))
print(df_read1.printSchema())
df_read1.show()

root
 |-- id: integer (nullable = true)
 |-- value: decimal(8,2) (nullable = true)

None
+---+--------+
| id|   value|
+---+--------+
|  2|98765.00|
|  1|12345.67|
|  3|98765.43|
+---+--------+



In [6]:
def test_partition_read(d1,d2):
    #print("spark.sql.parquet.enableVectorizedReader -" , spark.conf.get("spark.sql.parquet.enableVectorizedReader"))
    print("Decimal test with read - ",d1," and write - ",d2)
    errorlog.append(f"Scenario - Decimal test with read - {d1} and write - {d2}")
    try:
        decimaltest(d1,d2)
    except:
        print("Failed - Decimal test with read - ",d1," and write - ",d2)
        errorlog.append(f"- Failed - Decimal test with read - {d1} and write - {d2}")
    #print("read both partitions with schema1")
    try:
        readpartitions(schema1)
    except Exception as e:
        print("Failed - Read with schema - ",d1)
        errorlog.append(f" - Failed - Read with schema - {d1}")
    #print("read both partitions with schema2")
    try:
        readpartitions(schema2)
    except Exception as e:
        print("Failed - Read with schema - ",d2)
        errorlog.append(f"- Failed - Read with schema - {d2}")
    try:
        readpartitions_mergeschema()
    except Exception as e:
        print(f"Failed - Read with merge schema - {d1} , {d2} ")
        errorlog.append(f"Failed - Read with merge schema - {d1} , {d2}")
    else: 
        errorlog.append(f"Success - Read with merge schema - {d1} , {d2}")

    try:
        readpartitions_withoutschema()
    except Exception as e:
        print(f"Failed - Read without merge schema - {d1} , {d2} ")
        errorlog.append(f"Failed - Read without merge schema - {d1} , {d2}")
        

In [7]:
#Tests fo different combinations
errorlog = []
test_partition_read((8,2),(8,2))
test_partition_read((8,2),(12,2))
test_partition_read((8,2),(20,2))
test_partition_read((8,2),(8,1))
test_partition_read((8,2),(8,3))

test_partition_read((12,2),(8,2))
test_partition_read((12,2),(12,2))
test_partition_read((12,2),(20,2))
test_partition_read((12,2),(12,1))
test_partition_read((12,2),(12,3))

test_partition_read((20,2),(8,2))
test_partition_read((20,2),(12,2))
test_partition_read((20,2),(20,2))
test_partition_read((20,2),(20,1))
test_partition_read((20,2),(20,3))

test_partition_read((8,2),(9,2))
test_partition_read((8,2),(9,3))
test_partition_read((8,2),(7,2))

test_partition_read((12,2),(13,2))
test_partition_read((12,2),(13,3))
test_partition_read((12,2),(11,2))

test_partition_read((20,2),(21,2))
test_partition_read((20,2),(21,3))
test_partition_read((20,2),(19,2))
print("\n\n------ Output error log ---------\n\n")
for e in errorlog:
    print(e)

Decimal test with read -  (8, 2)  and write -  (8, 2)


+---+--------+----+
| id|   value|part|
+---+--------+----+
|  2|98765.00|   1|
|  1|12345.67|   1|
|  5|98765.43|   2|
|  6|98765.43|   2|
|  4|12345.68|   2|
|  3|98765.43|   1|
+---+--------+----+

+---+--------+----+
| id|   value|part|
+---+--------+----+
|  2|98765.00|   1|
|  1|12345.67|   1|
|  5|98765.43|   2|
|  6|98765.43|   2|
|  4|12345.68|   2|
|  3|98765.43|   1|
+---+--------+----+

+---+--------+----+
| id|   value|part|
+---+--------+----+
|  2|98765.00|   1|
|  1|12345.67|   1|
|  5|98765.43|   2|
|  6|98765.43|   2|
|  4|12345.68|   2|
|  3|98765.43|   1|
+---+--------+----+

+---+--------+----+
| id|   value|part|
+---+--------+----+
|  2|98765.00|   1|
|  1|12345.67|   1|
|  5|98765.43|   2|
|  6|98765.43|   2|
|  4|12345.68|   2|
|  3|98765.43|   1|
+---+--------+----+

Decimal test with read -  (8, 2)  and write -  (12, 2)
+---+--------+----+
| id|   value|part|
+---+--------+----+
|  4|12345.68|   2|
|  6|98765.43|   2|
|  5|98765.43|   2|
|  2|98765.00|   1|
|

In [8]:
readpartitions_mergeschema()

+---+--------+----+
| id|   value|part|
+---+--------+----+
|  1|12345.67|   1|
|  2|98765.00|   1|
|  6|98765.43|   2|
|  4|12345.68|   2|
|  5|98765.43|   2|
|  3|98765.43|   1|
+---+--------+----+



In [27]:
spark.conf.get("spark.sql.parquet.enableVectorizedReader")

'false'

In [39]:
test_partition_read((14,2),(15,3))

Decimal test with read -  (14, 2)  and write -  (15, 3)
+---+---------------+----+
| id|          value|part|
+---+---------------+----+
|  1|123456789012.67|   1|
|  2|987656789012.00|   1|
|  4|123456789012.68|   2|
|  6|987656789012.43|   2|
|  5|987656789012.43|   2|
|  3|987656789012.43|   1|
+---+---------------+----+

+---+----------------+----+
| id|           value|part|
+---+----------------+----+
|  1|123456789012.670|   1|
|  2|987656789012.000|   1|
|  4|123456789012.678|   2|
|  6|987656789012.432|   2|
|  5|987656789012.432|   2|
|  3|987656789012.430|   1|
+---+----------------+----+

read partition with merge schema exception
Failed - Read with merge schema - (14, 2) , (15, 3) 
+---+---------------+----+
| id|          value|part|
+---+---------------+----+
|  1|123456789012.67|   1|
|  2|987656789012.00|   1|
|  4|123456789012.68|   2|
|  6|987656789012.43|   2|
|  5|987656789012.43|   2|
|  3|987656789012.43|   1|
+---+---------------+----+



readpartitions(schema2)

In [37]:
schema3 = StructType([
        StructField("id", IntegerType(), False),
        StructField("value", DecimalType(14, 3), False)
    ])

readpartitions(schema3)

+---+-----+
| id|value|
+---+-----+
+---+-----+



In [44]:
# Verify the written parquet files   .schema(schema1)  .option("mergeSchema", "true")
df_read1 = spark.read.parquet("test/part=1")
df_read1 = df_read1.withColumn("value", df_read1["value"].cast(DecimalType(12, 0)))
print(df_read1.printSchema())
df_read1.show()

root
 |-- id: integer (nullable = true)
 |-- value: decimal(12,0) (nullable = true)

None
+---+------------+
| id|       value|
+---+------------+
|  1|123456789013|
|  2|987656789012|
|  3|987656789012|
+---+------------+

